In [18]:
import pandas as pd
import h5py
import os
import numpy as np
from selenobot.utils import DATA_DIR

In [2]:
print('Path exists:', os.path.exists('/home/prichter/Downloads/RS_GCF_000238295.3_embedding.h5'))
# example_embedding_path = '/home/prichter/Downloads/RS_GCF_000238295.3_embedding.h5'
example_embedding_path = '/home/prichter/Downloads/GB_GCA_019232935.1_embedding.h5'

Path exists: True


In [3]:
# What to Josh's GTDB embedding files look like?
store = pd.HDFStore(example_embedding_path, 'r')

In [24]:
os.path.join(DATA_DIR, 'val.csv')

'/home/prichter/Documents/selenobot/selenobot/../data/val.csv'

In [23]:
df = pd.read_csv(os.path.join(DATA_DIR, 'val.csv'), index_col=0)
df = df.set_index('id')
df.index.name = 'gene_id'
print(df)
df.to_csv(os.path.join(DATA_DIR, 'val.csv'))

           n_sec  n_trunc                                                seq  \
gene_id                                                                        
F1SR90[1]    1.0      2.0  MEQDGDQLSARPALETEGLRFLHVTVGSLLATYGWYIVFCCILLYV...   
P70551[1]    2.0    137.0  MGLLSVDLLITLQILPVFFSNCLFLALYDSVILLKHVALLLSRSKS...   
Q2KI76[1]    1.0      2.0  MERDGDQLSARPTLETEGLRFLHVTVGSLLATYGWYIVFSCILLYV...   
Q8VHV8[1]    1.0      2.0  MDRGEEPMSARPALETESLRFLHVTVGSLLASYGWYILFSCVLLYI...   
Q92813[1]    2.0    141.0  MGILSVDLLITLQILPVFFSNCLFLALYDSVILLKHVVLLLSRSKS...   
...          ...      ...                                                ...   
Q27YE2       NaN      NaN  MGQNQSRDKQKAIQNQPKDTGNRADIIPDATGMGPEFCKSCWFERR...   
Q6IVU5       NaN      NaN  MGNCNGASKSNQPDSSRVTQPAAEFRRVAHSSLYGRYNCKCCWFAD...   
Q6IUF9       NaN      NaN  MGNCNKPPKRPPNTQTSAAQPSAEFRRTALPSLYGRYNCKCCWFAD...   
Q27YE6       NaN      NaN  MGQKPSKPKAPPTTYESPRSSLTPDATGFGPEFCKSCWFERKGLIK...   
B2MW50       NaN      NaN  MGLRYSKAVKDKY

In [5]:
store.close()

In [16]:
with h5py.File(example_embedding_path, 'r') as f:
    keys = list(f.keys())
    dataset = f[keys[0]]
    embedding = np.empty(data.shape)
    dataset.read_direct(embedding)

In [17]:
embedding

array([-0.03593176,  0.03565327, -0.00251765, ...,  0.04481001,
       -0.02606017,  0.00906604])

In [7]:
[key.split(' # ')[0].replace('_', '.', 1) for key in keys]

['JAFAMZ010000001.1_1',
 'JAFAMZ010000001.1_2',
 'JAFAMZ010000001.1_3',
 'JAFAMZ010000001.1_4',
 'JAFAMZ010000001.1_5',
 'JAFAMZ010000002.1_1',
 'JAFAMZ010000002.1_2',
 'JAFAMZ010000003.1_1',
 'JAFAMZ010000003.1_2',
 'JAFAMZ010000003.1_3',
 'JAFAMZ010000003.1_4',
 'JAFAMZ010000004.1_1',
 'JAFAMZ010000004.1_10',
 'JAFAMZ010000004.1_2',
 'JAFAMZ010000004.1_3',
 'JAFAMZ010000004.1_4',
 'JAFAMZ010000004.1_5',
 'JAFAMZ010000004.1_6',
 'JAFAMZ010000004.1_7',
 'JAFAMZ010000004.1_8',
 'JAFAMZ010000004.1_9',
 'JAFAMZ010000005.1_1',
 'JAFAMZ010000005.1_2',
 'JAFAMZ010000005.1_3',
 'JAFAMZ010000006.1_1',
 'JAFAMZ010000006.1_2',
 'JAFAMZ010000006.1_3',
 'JAFAMZ010000006.1_4',
 'JAFAMZ010000006.1_5',
 'JAFAMZ010000006.1_6',
 'JAFAMZ010000006.1_7',
 'JAFAMZ010000006.1_8',
 'JAFAMZ010000006.1_9',
 'JAFAMZ010000007.1_1',
 'JAFAMZ010000007.1_2',
 'JAFAMZ010000007.1_3',
 'JAFAMZ010000008.1_1',
 'JAFAMZ010000008.1_2',
 'JAFAMZ010000008.1_3',
 'JAFAMZ010000008.1_4',
 'JAFAMZ010000008.1_5',
 'JAFAMZ0100000